In [1]:
# loading the book called "the verdict"

with open('the-verdict.txt', 'r', encoding='utf-8') as f:
  raw_text = f.read()

print(raw_text[:50])


I HAD always thought Jack Gisburn rather a cheap g


In [2]:
import tiktoken

In [3]:
# tokenizer BPE
tokenizer = tiktoken.get_encoding('gpt2')

In [4]:
total_chars = len(raw_text)

total_tokens = len(tokenizer.encode(raw_text))

print(f'Total characters: {total_chars}')
print(f"Tokens: {total_tokens}")

Total characters: 20479
Tokens: 5145


In [5]:
# tokenized_text[:50]

In [6]:
# implementing Dataset and Dataloader
import torch
from torch.utils.data import Dataset, DataLoader

class GPTDataset(Dataset):
  def __init__(self, text, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

    token_ids = tokenizer.encode(text, allowed_special = {"<|endoftext|>"})

    for i in range(0, len(token_ids) - max_length, stride):
      input_chunk = token_ids[i : i + max_length]
      target_chunk = token_ids[i + 1 : i + max_length + 1]

      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx], self.target_ids[idx]

In [7]:
# implementing the DataLoader

def create_dataloader(txt, batch_size=4, max_length=256, stride = 128,
                      shuffle = True, drop_last = True, num_workers = 0):

  tokenizer = tiktoken.get_encoding('gpt2')

  dataset = GPTDataset(txt, tokenizer, max_length, stride)

  dataloader = DataLoader(
      dataset,
      batch_size = batch_size,
      shuffle = shuffle,
      drop_last = drop_last,
      num_workers = num_workers
      )

  return dataloader

In [8]:
# GPT model's configuration

GPT_CONFIG_124M = {
    'vocab_size': 50257,
    'context_length': 256,
    'emd_dim': 768,
    'n_heads': 12,
    'n_layers': 12,
    'drop_rate': 0.1,
    'qkv_bias': False

}

In [9]:
# splitting the dataset into train and validation set

train_ratio = 0.9
split_idx = int(train_ratio * len(raw_text))

train_text = raw_text[:split_idx]
val_text = raw_text[split_idx:]

torch.manual_seed(123)

# creating train data loader
train_loader = create_dataloader(
    train_text,
    batch_size = 2,
    max_length = GPT_CONFIG_124M['context_length'],
    stride = GPT_CONFIG_124M['context_length'],
    shuffle = True,
    drop_last = True,
    num_workers = 0
)

# creating a validation dataloader

val_loader = create_dataloader(
    val_text,
    batch_size = 2,
    max_length = GPT_CONFIG_124M['context_length'],
    stride = GPT_CONFIG_124M['context_length'],
    shuffle = False,
    drop_last = False,
    num_workers = 0
)

In [10]:
input, target = train_loader.dataset[0]
print(f" input: {input}")

 input: tensor([   40,   367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138,
          257,  7026, 15632,   438,  2016,   257,   922,  5891,  1576,   438,
          568,   340,   373,   645,  1049,  5975,   284,   502,   284,  3285,
          326,    11,   287,   262,  6001,   286,   465, 13476,    11,   339,
          550,  5710,   465, 12036,    11,  6405,   257,  5527, 27075,    11,
          290,  4920,  2241,   287,   257,  4489,    64,   319,   262, 34686,
        41976,    13,   357, 10915,   314,  2138,  1807,   340,   561,   423,
          587, 10598,   393, 28537,  2014,   198,   198,     1,   464,  6001,
          286,   465, 13476,     1,   438,  5562,   373,   644,   262,  1466,
         1444,   340,    13,   314,   460,  3285,  9074,    13, 46606,   536,
         5469,   438, 14363,   938,  4842,  1650,   353,   438,  2934,   489,
         3255,   465, 48422,   540,   450,    67,  3299,    13,   366,  5189,
         1781,   340,   338,  1016,   284,  3758,   262,

In [11]:
print(f"target: {target}")

target: tensor([  367,  2885,  1464,  1807,  3619,   402,   271, 10899,  2138,   257,
         7026, 15632,   438,  2016,   257,   922,  5891,  1576,   438,   568,
          340,   373,   645,  1049,  5975,   284,   502,   284,  3285,   326,
           11,   287,   262,  6001,   286,   465, 13476,    11,   339,   550,
         5710,   465, 12036,    11,  6405,   257,  5527, 27075,    11,   290,
         4920,  2241,   287,   257,  4489,    64,   319,   262, 34686, 41976,
           13,   357, 10915,   314,  2138,  1807,   340,   561,   423,   587,
        10598,   393, 28537,  2014,   198,   198,     1,   464,  6001,   286,
          465, 13476,     1,   438,  5562,   373,   644,   262,  1466,  1444,
          340,    13,   314,   460,  3285,  9074,    13, 46606,   536,  5469,
          438, 14363,   938,  4842,  1650,   353,   438,  2934,   489,  3255,
          465, 48422,   540,   450,    67,  3299,    13,   366,  5189,  1781,
          340,   338,  1016,   284,  3758,   262,  1988,

In [12]:
# sanity check

if total_tokens * (train_ratio) < GPT_CONFIG_124M['context_length']:
  print("Not enough tokens for the training loader, "
  "Try to lower the GPT_CONFIG_124M['context_length] or "
  "increase the train_ratio")
else:
  print("Enough tokens for the training loader")

if total_tokens * (1 - train_ratio) < GPT_CONFIG_124M['context_length']:
  print("Not enough tokens for the validation loader, "
  "Try to lower the GPT_CONFIG_124M['context_length] or "
  "decrease the train_ratio")
else:
  print("Enough tokens for the validation loader")

Enough tokens for the training loader
Enough tokens for the validation loader


In [13]:
print('Train loader')
for x, y in train_loader:
  print(x.shape, y.shape)

print(f"Number of training set: {len(train_loader)}")

print('\nValidation loader')
for x, y in val_loader:
  print(x.shape, y.shape)

print(f"Number of validation set: {len(val_loader)}")

Train loader
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
Number of training set: 9

Validation loader
torch.Size([2, 256]) torch.Size([2, 256])
Number of validation set: 1


In [14]:
train_tokens = 0

for input_tokens, target_tokens in train_loader:
  train_tokens += input_tokens.numel()

print(f"Number of training tokens: {train_tokens}")

val_tokens = 0

for input_tokens, target_tokens in val_loader:
  val_tokens += input_tokens.numel()

print(f"Number of validation tokens: {val_tokens}")
print(f"All tokens: {train_tokens + val_tokens}")

Number of training tokens: 4608
Number of validation tokens: 512
All tokens: 5120


In [15]:
from torch import nn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, n_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % n_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.n_heads = n_heads
        self.head_dim = d_out // n_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.n_heads, self.head_dim) # Changed from num_heads to n_heads
        values = values.view(b, num_tokens, self.n_heads, self.head_dim) # Changed from num_heads to n_heads
        queries = queries.view(b, num_tokens, self.n_heads, self.head_dim) # Changed from num_heads to n_heads

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [16]:
class LayerNorm(nn.Module):
  def __init__(self, emd_dim):
    super().__init__()
    self.eps = 1e-5
    self.scale = nn.Parameter(torch.ones(emd_dim))
    self.shift = nn.Parameter(torch.zeros(emd_dim))

  def forward(self, x):
    mean = x.mean(dim = -1, keepdim = True)
    var = x.var(dim = -1, keepdim = True, unbiased = False)
    norm_x = (x - mean) / torch.sqrt(var + self.eps)
    # The model might lose important representational flexibility
    # if every normalized vector is constrained to always have zero mean and unit variance.
    #scale (also called gamma) is a learnable parameter that rescales each feature.
    #shift (also called beta) is a learnable parameter that shifts each feature.
    return self.scale * norm_x + self.shift

class GELU(nn.Module):
  def __init__(self):
    super().__init__()

  def forward(self, x):
    return 0.5 * x * (1 + torch.tanh(torch.sqrt(torch.tensor(2 / torch.pi)) * (x + 0.044715 * torch.pow(x, 3)))) # Converted torch.pi to tensor

class FeedForward(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.layers = nn.Sequential(
        nn.Linear(cfg['emd_dim'], 4 * cfg['emd_dim']),
        GELU(),
        nn.Linear(4 * cfg['emd_dim'], cfg['emd_dim'])
    )

  def forward(self, x):
    return self.layers(x)

In [17]:
class DeepNeuralNetwork(nn.Module):
    def __init__(self, layer_sizes, use_shortcut):
        super().__init__()
        self.use_shortcut = use_shortcut
        self.layers = nn.ModuleList([
            nn.Sequential(nn.Linear(layer_sizes[0], layer_sizes[1]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[1], layer_sizes[2]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[2], layer_sizes[3]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[3], layer_sizes[4]), GELU()),
            nn.Sequential(nn.Linear(layer_sizes[4], layer_sizes[5]), GELU())
        ])

    def forward(self, x):
        for layer in self.layers:
            # Compute the output of the current layer
            layer_output = layer(x)
            # Check if shortcut can be applied
            if self.use_shortcut and x.shape == layer_output.shape:
                x = x + layer_output
            else:
                x = layer_output
        return x


In [18]:
class TransformerBlock(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.att = MultiHeadAttention(
        d_in = cfg['emd_dim'],
        d_out = cfg['emd_dim'],
        # embed_dim = cfg['emd_dim'],
        context_length = cfg['context_length'],
        n_heads = cfg['n_heads'],
        dropout = cfg['drop_rate'], # Changed from drop_rate to dropout
        qkv_bias = cfg['qkv_bias'],
        # batch_first= True, # Removed batch_first as it's not a parameter in MultiHeadAttention
    )
    self.ff = FeedForward(cfg)
    self.ln1 = LayerNorm(cfg['emd_dim'])
    self.ln2 = LayerNorm(cfg['emd_dim'])
    self.drop_shortcut = nn.Dropout(cfg['drop_rate'])

  def forward(self, x):
    shortcut = x
    x = self.ln1(x)
    x= self.att(x) # batch, n_tokens, emd_dim
    x = self.drop_shortcut(x)
    x = x + shortcut # add the original input back

    shortcut = x
    x = self.ln2(x)
    x = self.ff(x)
    x = self.drop_shortcut(x)
    x = x + shortcut

    return x

In [19]:
# whole GPT-2 model

class GPTModel(nn.Module):
  def __init__(self, cfg):
    super().__init__()
    self.tok_emb = nn.Embedding(cfg['vocab_size'], cfg['emd_dim'])
    self.pos_emb = nn.Embedding(cfg['context_length'], cfg['emd_dim'])
    self.drop_emb = nn.Dropout(cfg['drop_rate'])

    self.trf_blocks = nn.Sequential(
        *[TransformerBlock(cfg) for _ in range(cfg['n_layers'])]
        )

    self.final_norm = LayerNorm(cfg['emd_dim'])

    self.out_head = nn.Linear(
        cfg['emd_dim'],
        cfg['vocab_size'],
        bias = False
    )

  def forward(self, in_idx):
    batch_size, seq_len = in_idx.shape
    tok_embeds = self.tok_emb(in_idx)
    pos_embeds = self.pos_emb(torch.arange(seq_len, device = in_idx.device))
    x = tok_embeds + pos_embeds # batch, token , emb_dim
    x = self.drop_emb(x)
    x = self.trf_blocks(x) # Corrected attribute name
    x = self.final_norm(x)
    logits = self.out_head(x)
    return logits



In [20]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval()

GPTModel(
  (tok_emb): Embedding(50257, 768)
  (pos_emb): Embedding(256, 768)
  (drop_emb): Dropout(p=0.1, inplace=False)
  (trf_blocks): Sequential(
    (0): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768, out_features=768, bias=False)
        (W_key): Linear(in_features=768, out_features=768, bias=False)
        (W_value): Linear(in_features=768, out_features=768, bias=False)
        (out_proj): Linear(in_features=768, out_features=768, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (ff): FeedForward(
        (layers): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU()
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
      (ln1): LayerNorm()
      (ln2): LayerNorm()
      (drop_shortcut): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerBlock(
      (att): MultiHeadAttention(
        (W_query): Linear(in_features=768

In [21]:
# loss funtion

def cal_loss_batch(input_batch, target_batch, model, device):
  input_batch, target_batch = input_batch.to(device), target_batch.to(device)
  logits = model(input_batch)
  loss = nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
  return loss

def cal_loss_loader(data_loader, model, device, num_batches = None):
  total_loss = 0
  if len(data_loader) == 0:
    return float('nan')
  elif num_batches is None:
    num_batches = len(data_loader)
  else:
    num_batches = min(num_batches, len(data_loader))

  for i, (input_batch, target_batch) in enumerate(data_loader):
    if i < num_batches:
      loss = cal_loss_batch(input_batch, target_batch, model, device)
      total_loss += loss.item()
    else:
      break
  return total_loss / num_batches


In [22]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [23]:
model.to(device)

torch.manual_seed(123)

with torch.no_grad():
  train_loss = cal_loss_loader(train_loader, model, device)
  val_loss = cal_loss_loader(val_loader, model, device)

print(f"Train loss: {train_loss:.4f}")
print(f"Validation loss: {val_loss:.4f}")


Train loss: 10.9876
Validation loss: 10.9811


In [26]:
perplexity = torch.exp(torch.tensor(train_loss))
perplexity

tensor(59135.2930)

In [33]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context

    ###Input batch:
 ###tensor([[6109, 3626, 6100,  345],
        ##[6109, 1110, 6622,  257]])

    for _ in range(max_new_tokens):

        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]

        # Get the predictions
        with torch.no_grad():
            logits = model(idx_cond) ### batch, n_tokens, vocab_size

        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1)

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [34]:
import tiktoken

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"



token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


In [27]:

# pretraining loop of the GPT model

In [32]:
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
  model.eval()

  with torch.no_grad():
    train_loss = cal_loss_loader(train_loader, model, device,num_batches = eval_iter)
    val_loss = cal_loss_loader(val_loader, model, device, num_batches = eval_iter)

  model.train()

  return train_loss, val_loss

In [36]:
def generate_and_print_sample(model, tokenizer, device, start_context):
  model.eval()
  context_size = model.pos_emb.weight.shape[0]
  encoded = text_to_token_ids(start_context, tokenizer).to(device)

  with torch.no_grad():
    token_ids = generate_text_simple(
        model= model, idx = encoded,
        max_new_tokens = 50, context_size = context_size
    )

  decoded_text = token_ids_to_text(token_ids, tokenizer)
  print(decoded_text.replace("\n", " "))  # Compact print format
  model.train()

In [37]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):

  train_losses, val_losses, track_tokens_seen = [], [], []
  tokens_seen, global_step = 0, -1

  for epoch in range(num_epochs):
    model.train()

    for input_batch, target_batch in train_loader:
      optimizer.zero_grad()
      loss = cal_loss_batch(input_batch, target_batch, model, device)
      loss.backward()
      optimizer.step()
      tokens_seen += input_batch.numel()
      global_step += 1

      if global_step % eval_freq == 0:
        train_loss, val_loss = evaluate_model(
            model, train_loader, val_loader, device, eval_iter
        )
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        track_tokens_seen.append(tokens_seen)
        print(f"Ep {epoch+1} (Step {global_step:06d}): "
              f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")


    generate_and_print_sample(
        model, tokenizer, device, start_context
    )
  return train_losses, val_losses, track_tokens_seen

In [38]:
# Note:
# Uncomment the following code to calculate the execution time
import time
start_time = time.time()

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)

# Note:
# Uncomment the following code to show the execution time
end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 9.783, Val loss 9.927
Ep 1 (Step 000005): Train loss 7.985, Val loss 8.335
Every effort moves you,,,,,,,,,,,,.                                     
Ep 2 (Step 000010): Train loss 6.753, Val loss 7.048
Ep 2 (Step 000015): Train loss 6.114, Val loss 6.573
Every effort moves you, and,, and, and,,,,, and, and,,,,,,,,,,,,,, and,,,, and,, and,,,,, and,,,,,,
Ep 3 (Step 000020): Train loss 5.525, Val loss 6.490
Ep 3 (Step 000025): Train loss 5.324, Val loss 6.387
Every effort moves you, and to the picture.                      "I, and the of the of the's the honour, and, and I had been, and I
Ep 4 (Step 000030): Train loss 4.761, Val loss 6.360
Ep 4 (Step 000035): Train loss 4.461, Val loss 6.258
Every effort moves you of the to the picture--as of the picture--as I had been " it was his " I was the     "I was his I had been the his pictures--and it the picture and I had been the picture of
Ep 5 (Step 000040): Train loss 3.833, Val loss 6.196
Every effort moves yo